# VERSAO DE DESENVOLVIMENTO, 2.0

In [9]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
from tensorflow import keras 
import seaborn as sns
import sklearn as sklearn
from sklearn.metrics import median_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
os.chdir('C:\Repos\DS-Apis-Globo')

## Função para carregar o arquivo de dados de tempo disponível para anuncios

In [10]:
def read_inv(file_inv):
    #file_inv = '../Data/tvaberta_inventory_availability.csv(1).csv'
    dateparser_inv = lambda x: datetime.strptime(x, '%d/%m/%Y')
    sep_inv = ';'
    parsedates_inv = ['date']
    df_inv = pd.read_csv(file_inv,
                         sep=sep_inv,
                         parse_dates=parsedates_inv,
                         date_parser=dateparser_inv)
    df_inv['weekday'] = df_inv.date.dt.dayofweek
    # acrescentando colunas do outro conjunto de dados de audiencia media e prevista para realizar mesclagem dos conjuntos
    # valores sao inicializados com -1, caso alguma requisicao retorne com -1 em audiencia prevista significa que a data do registro correspondeu apenas à este conjunto df_inv.
    df_inv['average_audience'] = -1
    df_inv['predicted_audience'] = -1
    return df_inv

## Função para carregar o arquivo de dados de media historica de anuncios


In [11]:
## Funcao para carregar o arquivo de dados de media historica de anuncios
def read_pro(file_pro, preserve = True):
    #file_pro = '../Data/tvaberta_program_audience(1).csv'
    dateparser_pro = lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    sep_pro = ','
    parsedates_pro = ['program_start_time']
    indexcol_pro = 'program_start_time'
    df_pro = pd.read_csv(file_pro,
                         sep=sep_pro,
                         parse_dates=parsedates_pro,
                         date_parser=dateparser_pro)
    df_pro['exhibition_date'] = pd.to_datetime(df_pro['exhibition_date'], format='%Y-%m-%d')
    df_pro = df_pro.rename(columns={"exhibition_date": "date"})
    df_pro['weekday'] = df_pro.date.dt.dayofweek  
    # inicializando audiencia prevista em -1
    df_pro['predicted_audience'] = -1
    # valores de tempo estimado inicializados com -1, caso alguma requisicao retorne com -1 em tempo estimado significa que a data do registro correspondeu apenas à este conjunto df_pro.
    df_pro['available_time'] = -1
    if not preserve:
        del df_pro['program_start_time']
    return df_pro

## Calculo da mediana que recebe como parametro um pandas dataframe, uma coluna alvo de calculo e uma janela temporal de calculo.


In [12]:
def median_window(df,target,window):
    if len(df[target])>0:
        # o calculo da mediana se da pelo valor na posicao central do conjunto ordenado portanto as linhas do dataframe sao ordenadas com base na coluna alvo, com complexidade n*log(2)*n em qualquer caso de ordenacao. Dado que o pandas utiliza o algoritmo merge sort na funcao .sort_values
        df.sort_values(by=[target])
        # captura-se o primeiro valor da coluna alvo do daframe
        fill = df[target].values[0]
        # se o tamanho do dataframe for par, deve-se calcular a mediana atraves da media do valor central e o proximo elemento, dentro da janela de calculo
        if window % 2 == 0:
            s1 = int(window / 2)
            s2 = int(window / 2) + 1
            # para otimizacão do calculo da mediana sao utilizados 2 vetores com a coluna de valores deslocados pelo tamanho da janela de calculo, que no caso deste exercicio tem tamanho 4. Portanto a mediana, de janela 4, para qualquer registro dentro do dataframe ordenado sera a media aritmetica entre o 2o e 3o valor anterior à este registro.
            laux1 = df[target].shift(s1,fill_value=fill)
            laux2 = df[target].shift(s2,fill_value=fill)
            return ((laux1 + laux2) / 2)
        else:
            s1 = int(window/2) + 1
            return df[target].shift(s1,fill_value=fill)

# Função para varrer o dataframe com os registros de audiencia e calcular a mediana para cada registro pertencente a uma localizacao, codigo de programa e dia da semana distinto.

In [13]:
def df_predicted_audience(df):
    # obtendo lista de localizacoes distintas de transmissao
    l_signal = df.signal.unique()
    # iteracao para cada localizacao
    for v_signal in l_signal:
        # filtrando o dataframe para a localizacao da iteracao
        df_aux = df.loc[df['signal'] == v_signal]
        # obtendo lista de programas distintos dentro do dataframe filtrado
        l_program_code = df_aux.program_code.unique()
        # iterando sobre cada programa
        for v_program in l_program_code:
            # filtrando o dataframe a localizacao e codigo de programa da iteracao
            df_aux = df.loc[(df['signal'] == v_signal) & (df['program_code'] == v_program)]
            # obtendo lista de dias da semana disponiveis dentro do dataframe filtrado
            l_weekdays = df_aux.weekday.unique()
            for v_day in l_weekdays:
                # obtendo um dataframe auxiliar com todos os registros da localizacao, codigo de programa e dia da semana filtrados
                df_aux = df.loc[(df['weekday'] == v_day) & (df['signal'] == v_signal) & (
                            df['program_code'] == v_program)]
                #df_aux.sort_values(by=['average_audience'])
                # acionando a funcao para o calculo da audiencia estimada por meio da mediana, dentro do dataframe auxiliar
                df_aux['predicted_audience'] = median_window(df_aux,'average_audience',4)
                # gravando os valores de audiencia estimada correspondentes aos indices dos registros filtrados no dataframe resultante.
                df.predicted_audience.loc[df_aux.index] = df_aux['predicted_audience'].values
    return df

## Carregando conjunto de dados

In [14]:
df_inv = read_inv('./Data/tvaberta_inventory_availability.csv(1).csv')
df_pro = read_pro('./Data/tvaberta_program_audience.csv')

## Preparando conjunto resultante

In [15]:
df_res = df_pro.copy()
df_res = df_predicted_audience(df_res)
df_res

 slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\dldou\.conda\envs\datascience\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\dldou\.conda\envs\datascience\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\dldou\.conda\envs\datascience\lib\site-packages\i

,signal,program_code,date,program_start_time,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,2020-06-05 15:01:00,9.338920e+05,4,9.338920e+05,-1
1,SP1,VALE,2020-06-24,2020-06-24 19:36:00,1.835375e+06,2,1.835375e+06,-1
2,BH,PTV1,2020-05-23,2020-05-23 15:01:00,2.874812e+05,5,2.874812e+05,-1
3,DF,AUTO,2020-05-03,2020-05-03 12:14:00,8.649481e+04,6,8.649481e+04,-1
4,DF,FGGE,2020-05-03,2020-05-03 18:46:00,1.521656e+05,6,1.521656e+05,-1
...,...,...,...,...,...,...,...,...
8982,NAC,N18H,2020-05-04,2020-05-04 21:31:00,2.093098e+07,0,2.018653e+07,-1
8983,NAC,JHOJ,2020-05-13,2020-05-13 16:25:00,1.331286e+07,2,1.318622e+07,-1
8984,NAC,SHOV,2020-06-07,2020-06-07 15:30:00,8.610939e+06,6,9.232443e+06,-1
8985,NAC,SHOV,2020-05-17,2020-05-17 15:30:00,1.056899e+07,6,8.947505e+06,-1


## Mesclando conjunto de audiencia e tempo disponivel por anuncio

In [9]:
df_res = pd.concat([df_res,df_inv])
df_res

,signal,program_code,date,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,9.338920e+05,4,9.338920e+05,-1
1,SP1,VALE,2020-06-24,1.835375e+06,2,1.835375e+06,-1
2,BH,PTV1,2020-05-23,2.874812e+05,5,2.874812e+05,-1
3,DF,AUTO,2020-05-03,8.649481e+04,6,8.649481e+04,-1
4,DF,FGGE,2020-05-03,1.521656e+05,6,1.521656e+05,-1
...,...,...,...,...,...,...,...
2287,BH,NBRA,2020-08-24,-1.000000e+00,0,-1.000000e+00,218
2288,BH,NBRA,2020-08-25,-1.000000e+00,1,-1.000000e+00,66
2289,BH,NBRA,2020-08-26,-1.000000e+00,2,-1.000000e+00,44
2290,BH,NBRA,2020-08-27,-1.000000e+00,3,-1.000000e+00,190


## Funções da REST-API

In [10]:
def get_predicted(_program_code,_date):
    return json.loads(df_res.loc[(df_res['date'] == _date) & (df_res['program_code'] == _program_code)][['available_time','predicted_audience']].to_json(orient="records"))

In [11]:
def get_period(begin,end):
    return json.loads(df_res.loc[(df_res['date'] >= begin) & (df_res['date'] <= end)][['program_code','available_time','predicted_audience']].to_json(orient="records"))

In [14]:
get_predicted('PTV1','2020-06-05')

[{'available_time': -1, 'predicted_audience': 933891.9638554216},
 {'available_time': -1, 'predicted_audience': 211398.8072289157},
 {'available_time': -1, 'predicted_audience': 997945.0060240964},
 {'available_time': -1, 'predicted_audience': 208046.9723311852},
 {'available_time': -1, 'predicted_audience': 285912.9795460913},
 {'available_time': -1, 'predicted_audience': 12325816.445783133}]

In [15]:
get_period('2020-05-03','2020-06-24')

 'SAME',
  'available_time': -1,
  'predicted_audience': 423477.6382978723},
 {'program_code': 'PTV1',
  'available_time': -1,
  'predicted_audience': 211398.8072289157},
 {'program_code': 'GLCO',
  'available_time': -1,
  'predicted_audience': 63191.4814814815},
 {'program_code': 'SHOV',
  'available_time': -1,
  'predicted_audience': 229360.7359243698},
 {'program_code': 'MEST',
  'available_time': -1,
  'predicted_audience': 474733.9605263158},
 {'program_code': 'BPRA',
  'available_time': -1,
  'predicted_audience': 78773.5033557047},
 {'program_code': 'TARA',
  'available_time': -1,
  'predicted_audience': 319404.2469765684},
 {'program_code': 'COR1',
  'available_time': -1,
  'predicted_audience': 413771.6666666667},
 {'program_code': 'JHOJ',
  'available_time': -1,
  'predicted_audience': 276943.9945054945},
 {'program_code': 'SAME',
  'available_time': -1,
  'predicted_audience': 423477.6382978723},
 {'program_code': 'PTV2',
  'available_time': -1,
  'predicted_audience': 19076

# DATA ANALYTICS COM AUDIENCE

In [18]:
df_res

,signal,program_code,date,program_start_time,average_audience,weekday,predicted_audience,available_time
0,SP1,PTV1,2020-06-05,2020-06-05 15:01:00,9.338920e+05,4,9.338920e+05,-1
1,SP1,VALE,2020-06-24,2020-06-24 19:36:00,1.835375e+06,2,1.835375e+06,-1
2,BH,PTV1,2020-05-23,2020-05-23 15:01:00,2.874812e+05,5,2.874812e+05,-1
3,DF,AUTO,2020-05-03,2020-05-03 12:14:00,8.649481e+04,6,8.649481e+04,-1
4,DF,FGGE,2020-05-03,2020-05-03 18:46:00,1.521656e+05,6,1.521656e+05,-1
...,...,...,...,...,...,...,...,...
8982,NAC,N18H,2020-05-04,2020-05-04 21:31:00,2.093098e+07,0,2.018653e+07,-1
8983,NAC,JHOJ,2020-05-13,2020-05-13 16:25:00,1.331286e+07,2,1.318622e+07,-1
8984,NAC,SHOV,2020-06-07,2020-06-07 15:30:00,8.610939e+06,6,9.232443e+06,-1
8985,NAC,SHOV,2020-05-17,2020-05-17 15:30:00,1.056899e+07,6,8.947505e+06,-1


In [20]:
locations = df_res.signal.unique()
locations

array(['SP1', 'BH', 'DF', 'RJ', 'PE1', 'NAC'], dtype=object)

In [27]:
for v_signal in locations:
    print(v_signal,df_res.loc[df_res['signal'] == v_signal].describe())
    print()

SP1        average_audience      weekday  predicted_audience  available_time
count      1.493000e+03  1493.000000        1.493000e+03          1493.0
mean       1.300319e+06     2.892163        1.296108e+06            -1.0
std        8.163674e+05     1.962046        8.044138e+05             0.0
min        2.211534e+05     0.000000        2.211534e+05            -1.0
25%        6.134414e+05     1.000000        6.246724e+05            -1.0
50%        1.093147e+06     3.000000        1.119399e+06            -1.0
75%        1.783313e+06     5.000000        1.774248e+06            -1.0
max        3.501991e+06     6.000000        3.343718e+06            -1.0

BH        average_audience      weekday  predicted_audience  available_time
count      1.507000e+03  1507.000000        1.507000e+03          1507.0
mean       3.878630e+05     2.911745        3.895503e+05            -1.0
std        3.017467e+05     1.963353        3.021670e+05             0.0
min        2.602719e+04     0.000000       

## maiores médias foram SP1 com 1.300319e+06 e RJ com 1.041062e+06

## verificando programas existentes em SP1

In [32]:
sp1_programs = df_res.loc[df_res['signal'] == 'SP1'].program_code.unique()
sp1_programs

array(['PTV1', 'VALE', 'NBRA', 'JNAS', 'GRUD', 'TARA', 'COR1', 'BPRA',
       'MALH', 'N19S', 'SSUP', 'JHOJ', 'HORA', 'DFAU', 'MEPR', 'N19H',
       'FATI', 'EMPR', 'DOMA', 'COR2', 'ANTE', 'TMAX', 'JGLO', 'MISA',
       'SAME', 'N18S', 'PTV2', 'HUCK', 'BIGB', 'BIAL', 'FANT', 'JNAC',
       'N18H', 'PT2S', 'CINE', 'ALTA', 'CBIL', 'N20S', 'CORO', 'TELA',
       'MBOA', 'N20H', 'SHT1', 'TNOB', 'AUTO', 'SUCI', 'SERA', 'CIMA',
       'REPO', 'CASA', 'SHO5', 'ACDA', 'SHOV', 'SSAB', 'COR3', 'FGGE',
       'ESPO', 'ZORR', 'PREG', 'ESP2', '_999', 'F1VT', 'PEGD', 'TECN',
       'VOIK', 'TECA', 'MEST', 'TECV', 'INES', 'CESP'], dtype=object)

## programas existentes em RJ

In [33]:
rj_programs = df_res.loc[df_res['signal'] == 'RJ'].program_code.unique()
rj_programs

array(['BIAL', 'MALH', 'FATI', 'N18S', 'SSAB', 'PTV2', 'JNAC', 'COR2',
       'N20H', 'ZORR', 'BIGB', 'N19H', 'HORA', 'SERA', 'COR1', 'ESPO',
       'PTV1', 'BPRA', 'NBRA', 'N18H', 'FANT', 'JHOJ', 'TARA', 'AUTO',
       'ALTA', 'EMPR', 'TELA', 'SHT1', 'PT2S', 'JGLO', 'SAME', 'MBOA',
       'SUCI', 'N19S', 'CIMA', 'CESP', 'TNOB', 'VALE', 'HUCK', 'JNAS',
       'GRUD', 'TMAX', 'GLCO', 'CORO', 'DFAU', 'TECV', 'MISA', 'DOMA',
       'REPO', 'N20S', 'CBIL', 'FGGE', 'CASA', 'COR3', 'SHOV', 'CINE',
       'MEST', 'SSUP', 'SHO5', '_999', 'MEPR', 'VOIK', 'INES', 'PEGD',
       'TECN', 'PREG', 'ESP2', 'TECA', 'F1VT', 'ACDA'], dtype=object)

## programas comuns entre SP1 e RJ

In [43]:
eq_programs = list(set(sp1_programs).intersection(rj_programs))

In [57]:
avg_list = []
for _program in eq_programs:
    sp1_mean = df_res.loc[ (df_res['signal'] == 'SP1') & (df_res['program_code'] == _program) ].average_audience.mean()
    rj_mean = df_res.loc[ (df_res['signal'] == 'RJ') & (df_res['program_code'] == _program) ].average_audience.mean()
    avg_list.append([ _program, sp1_mean, rj_mean, (sp1_mean+rj_mean)/2 ])

In [58]:
avg_list

[['COR1', 424722.51112757757, 338926.41652086464, 381824.4638242211],
 ['COR2', 330036.68201425293, 269366.99874621804, 299701.84038023546],
 ['CASA', 490026.5279302192, 422821.247135523, 456423.8875328711],
 ['SHO5', 1075173.2235346385, 886706.2435221784, 980939.7335284085],
 ['N20S', 2836473.808180557, 2101812.21268561, 2469143.0104330834],
 ['SSAB', 1064396.3272989232, 849660.2412654649, 957028.284282194],
 ['SSUP', 1334576.1663553098, 948788.6698123522, 1141682.418083831],
 ['MISA', 392161.7039339684, 378173.0225550199, 385167.36324449413],
 ['VALE', 1915397.1800546288, 1613695.5339557142, 1764546.3570051715],
 ['JNAS', 2542515.6784271053, 1918321.7144854816, 2230418.6964562936],
 ['PREG', 871850.9702380953, 695028.534085213, 783439.7521616542],
 ['TECA', 2512437.8038461534, 1793046.6403846154, 2152742.2221153844],
 ['PTV2', 2150494.730641602, 1812750.7336550886, 1981622.7321483453],
 ['MALH', 1659183.6956711216, 1497299.335611332, 1578241.515641227],
 ['N19S', 2414673.7876725434, 

In [ ]:
for v_program in v_programs:
    print(v_signal, df_res.loc[df_res['signal'] == 'SP1' & df_res['signal'] == 'SP1'])
    print()